In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import string
from nltk.corpus import stopwords 
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
import emoji

In [2]:
df = pd.read_csv('../mi-person/data/go_emotions.csv')

In [17]:
# df.shape
# df.sample(7000)

In [4]:
df1 = df[df['amusement'] == 1]
df1.shape

(9245, 31)

In [5]:
df2 = df.drop(columns=['id', 'text', 'example_very_unclear'])
df_concat = pd.DataFrame()
for column in df2.columns:
    df_temp = df[df[column] == 1].sample(673)
    
    df_concat = pd.concat([df_concat, df_temp])

In [6]:
df_concat.shape

(18844, 31)

In [7]:
X_train = df_concat[['text']]
X_train.head()

,text
153353,"Ya, he’s a lot better. If someone maxes dlo th..."
7372,Usually they dont even show up when you call t...
99188,Thank you! That's the nicest thing I heard tod...
21114,"Geezuz, this brute is damn hot. I'm In Love."
141343,Even while [NAME] in a America are discriminat...


In [8]:

def __remove_punctuation(text):
    """
        remove punctuation from text and lower case it
    """
    text = str(text)

    punctuations = string.punctuation
    punctuations += '“'
    punctuations += '’'
    punctuations += '”'
    punctuations += '’'
    punctuations += ' — '
    punctuations += 'â€œ'
    punctuations += 'â€¦'
    punctuations += 'â€'
    punctuations += '€™'
    punctuations += '€'
    punctuations += '™'
    punctuations += '¦'
    punctuations += 'œ'
    punctuations += 'Â'
    punctuations += 'Ã'
    punctuations += '— '
    punctuations += '¶'
    punctuations += '§'
    punctuations += '£'
    punctuations += '©'
    punctuations += 'ª'
    punctuations += '³'

    # text = emoji.get_emoji_regexp().sub(u'', text)

    for punctuation in punctuations:
        text = text.replace(punctuation, ' ') 
        #text = text.replace('donald', 'trump')
        #text = text.replace('clinton', 'hillary')
    return text.lower() # lower case

def __remove_numbers(text):
    """
        remove number from text
    """
    text = str(text)

    words_only = ''.join([i for i in text if not i.isdigit()])
    return words_only

def __remove_stopwords(text):
    """
        remove stop words from text
    """
    text = str(text)

    stop_words = stopwords.words('english')
    #stop_words += stopwords.words('portuguese')
    stop_words.append('mr')
    stop_words = set(stop_words)

    tokenized = word_tokenize(text)
    without_stopwords = [word for word in tokenized if not word in stop_words]
    return without_stopwords

def __lemmatize(text):
    """
        lemmatize text
    """
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(word) for word in text]
    lemmatized_string = " ".join(lemmatized)
    return lemmatized_string


def process_data(df):
    """
        process the data
    """

    df_ = df.copy()
        
    df_['text'] = df_['text'].apply(__remove_punctuation)

    df_['text'] = df_['text'].apply(__remove_numbers)

    df_['text'] = df_['text'].apply(__remove_stopwords)

    df_['text'] = df_['text'].apply(__lemmatize)
    
    return df_

In [19]:
cleaned_sentences = process_data(X_train)
cleaned_sentences.shape

(18844, 1)

In [10]:
# cleaned_sentences = [basic_cleaning(sentence) for sentence in X_train]

In [37]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf_vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'))
processed_features = tf_idf_vectorizer.fit_transform(cleaned_sentences)


In [38]:
# weighted_words = pd.DataFrame(tf_idf_vectorizer.fit_transform(processed_features).toarray(),
#                  columns = tf_idf_vectorizer.get_feature_names_out())

# weighted_words

In [39]:
tf_idf_summariser = TfidfVectorizer()

    # fit TF-IDF to data and transform it
X_tf_idf_summary = tf_idf_summariser.fit_transform(cleaned_sentences)

# create a dataframe with words and its tf-idf values for every sentence (row)
df_tf_idf_summary = pd.DataFrame(data = X_tf_idf_summary.toarray(),
                                     columns = tf_idf_summariser.get_feature_names())


/home/tatchiwiggers/.pyenv/versions/3.8.12/envs/mi-person/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [40]:
# processed_features = processed_features.toarray()
# processed_features.shape
df_tf_idf_summary

,text
0,1.0


In [32]:
y_train = df_concat.drop(columns=['id', 'text', 'example_very_unclear'])
y_train.shape

(18844, 28)

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

In [27]:
model = OneVsRestClassifier(LogisticRegression()).fit(processed_features, y_train)
# model = OneVsRestClassifier(LogisticRegression()).fit(processed_features, y_train)

ValueError: Found input variables with inconsistent numbers of samples: [1, 18844]

In [ ]:
df['text'].iloc[2]


"You do right, if you don't care then fuck 'em!"

In [ ]:
processed_test = vectorizer.transform(df['text'].values)
prediction = model.predict(processed_test)
prediction_proba = model.predict_proba(processed_test)

NameError: name 'model' is not defined

In [ ]:
#print(prediction[0])
print(prediction_proba[0].sum())
print(prediction_proba[0][np.argmax(prediction[2])]*100)
np.argmax(prediction[2])

1.5879333545366263
4.139329719007467


2

In [ ]:
processed_features.shape

(1, 1)

In [ ]:
# data = {"text": ["that game hurt"]}


In [ ]:
# X_test = pd.DataFrame.from_dict(data)
# test_vec = vectorizer.transform(X_test['text'].values)


In [ ]:
model.predict(processed_features)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
model.predict_proba(processed_features)

array([[0.44739292, 0.03586595, 0.02809953, ..., 0.02414154, 0.02709744,
        0.03835104],
       [0.058321  , 0.24651841, 0.06306535, ..., 0.05902671, 0.02587264,
        0.04027202],
       [0.10469759, 0.0209913 , 0.05397462, ..., 0.03384042, 0.02801978,
        0.04331961],
       ...,
       [0.04203796, 0.04283444, 0.0555304 , ..., 0.03986793, 0.01540153,
        0.09163799],
       [0.10061908, 0.03004642, 0.04698681, ..., 0.02987135, 0.01896844,
        0.0306477 ],
       [0.04314211, 0.03290709, 0.04596256, ..., 0.0680064 , 0.04222355,
        0.11853829]])

In [ ]:
model.score(processed_features, y_train)*100

7.99724050095521